In [1]:
# Import numpy in order to correctly read test data
import numpy as np

# Import urllib in order to download test data from Github repo
import urllib

# Append location of netrics module base directory to system path
# NOTE: only required if permanent install not made (see comments above)
import sys
sys.path.append('/Users/bgraham/Dropbox/Sites/software/netrics/')

# Load netrics module
import netrics as netrics

In [4]:
download =  '/Users/bgraham/Dropbox/sites/software/netrics/Notebooks/'

In [5]:
#url = 'https://github.com/bryangraham/netrics/blob/master/Notebooks/Nyakatoke_Example.npz'
#urllib.urlretrieve(url, download + "Nyakatoke_Example.npz")



NyakatokeTestDataset = np.load(download + "Nyakatoke_Example.npz")

# Extract adjacency matrix
D = NyakatokeTestDataset['D']

# Initialize list of dyad-specific covariates as elements
# W = [W0, W1, W2,...WK-1]
W = []

# Initialize list with covariate labels
cov_names = []

for matrix in NyakatokeTestDataset.files:
    if matrix != 'D':
        W.append(NyakatokeTestDataset[matrix])
        cov_names.append(matrix)   

In [ ]:
help(netrics.tetrad_logit)

In [ ]:
[beta_TL, vcov_beta_TL, tetrad_frac_TL, success] = \
    netrics.tetrad_logit(D, W, dtcon=None, silent=False, W_names=cov_names)